In [1]:
# Dependences
import os
import json
import requests as req
import plotly.express as px
import pandas as pd
from sqlalchemy import create_engine, Table, Column, Integer, String, DateTime, Numeric, MetaData
from time import time

# Prepare SQL connection string to be used on the functions
CONN_STRING_PATH = '../config/sentinel-conn-string.txt'
CONN_STRING_PATH_ETL = '../config/etl_connection_string.txt'

with open(CONN_STRING_PATH, 'r') as fid:
    conn_string = fid.read()
    
with open(CONN_STRING_PATH_ETL, 'r') as fid:
    conn_string_etl = fid.read()
    
connection = create_engine(conn_string, pool_recycle=3600).connect()

## bulk insert requres some "executemany_*" arguments
connection_etl = create_engine(
     conn_string_etl
    ,pool_recycle=3600
    ,executemany_mode='values'
    ,executemany_values_page_size=10000
    ).connect()

## Chain Economics Table

Supports:

- relative token distribution

In [2]:
QUERY = """
        SELECT s.*
            ,(s.circulating_fil +
              s.vested_fil +
              s.mined_fil +
              s.burnt_fil +
              s.locked_fil) AS supply_fil
        FROM
        (
            SELECT 
                to_timestamp(b.timestamp) AS timestamp
                ,AVG(ce.circulating_fil::NUMERIC / 1e18) AS circulating_fil
                ,AVG(ce.vested_fil::NUMERIC / 1e18) AS vested_fil
                ,AVG(ce.mined_fil::NUMERIC / 1e18) AS mined_fil
                ,AVG(ce.burnt_fil::NUMERIC / 1e18) AS burnt_fil
                ,AVG(ce.locked_fil::NUMERIC / 1e18) AS locked_fil
                ,MAX(ce.parent_state_root) AS parent_state_root

            FROM chain_economics ce
            LEFT JOIN block_headers b 
                ON b.parent_state_root = ce.parent_state_root 
            
            GROUP BY to_timestamp(b.timestamp)
        ) s
            """

df = ( pd.read_sql(QUERY, connection) )
df.head(10)

,timestamp,circulating_fil,vested_fil,mined_fil,burnt_fil,locked_fil,parent_state_root,supply_fil
0,2020-09-29 16:02:00+00:00,0.0,23039.999638,4.032795e+06,2.048352e+06,8.530596e+06,bafy2bzacec4tfbnspldhfffcar5a5ygbn4q3ppddvym7h...,1.463478e+07
1,2020-09-29 16:02:30+00:00,0.0,23039.999638,4.032872e+06,2.048352e+06,8.530707e+06,bafy2bzaced53lneuiuu44th4ljqxwtslvzvqjfit5hjgp...,1.463497e+07
2,2020-09-29 16:03:00+00:00,0.0,23039.999638,4.032950e+06,2.048352e+06,8.530834e+06,bafy2bzaceddmqmeq3rlge3gd4mabjr2si24snlm7e7t6w...,1.463518e+07
3,2020-09-29 16:03:30+00:00,0.0,23039.999638,4.032989e+06,2.048353e+06,8.530910e+06,bafy2bzaceclqxtegqcfiablhia5rdr4p6jge55frajdqq...,1.463529e+07
4,2020-09-29 16:04:00+00:00,0.0,23039.999638,4.033080e+06,2.048353e+06,8.531032e+06,bafy2bzaceaximbvgwvqqqgg2s6mkitcn2lglciaolqgr4...,1.463550e+07
5,2020-09-29 16:04:30+00:00,0.0,23039.999638,4.033106e+06,2.048353e+06,8.531086e+06,bafy2bzaceatjhpzevbv2ysf2ftbkzxfvamy55m3dpfcdt...,1.463558e+07
6,2020-09-29 16:05:00+00:00,0.0,23039.999638,4.033184e+06,2.048353e+06,8.531214e+06,bafy2bzacebvakeudyux53wemg2sweomm2sg5gsivxw2qy...,1.463579e+07
7,2020-09-29 16:05:30+00:00,0.0,23039.999638,4.033262e+06,2.048353e+06,8.531324e+06,bafy2bzacecw3vj2aztha75efh3htacchgsrzhwcpb7mxe...,1.463598e+07
8,2020-09-29 16:06:00+00:00,0.0,23039.999638,4.033352e+06,2.048354e+06,8.531448e+06,bafy2bzacedogseq4xdvkll44vekghnzucahhx7lshi7cp...,1.463619e+07
9,2020-09-29 16:06:30+00:00,0.0,23039.999638,4.033430e+06,2.048354e+06,8.531550e+06,bafy2bzaceaevjrczgqkeafpmmocwm6evmlptgtbbnz3be...,1.463637e+07


In [3]:
meta = MetaData()

staging_token_distribution = Table(
    'staging_chain_economics', meta, 
    Column('id', Integer, primary_key = True),
    Column('parent_state_root', String),
    Column('timestamp', DateTime), 
    Column('circulating_fil', Numeric), 
    Column('vested_fil', Numeric),
    Column('mined_fil', Numeric),
    Column('burnt_fil', Numeric),
    Column('locked_fil', Numeric), 
    Column('supply_fil', Numeric),
 )
meta.create_all(connection_etl)

In [4]:
df.to_sql(
     name="staging_chain_economics"
    ,schema="public"
    ,con=connection_etl
    ,if_exists="append"
    ,index=False
    ,chunksize = 10000
    )

## Chain Powers

Supports:

- Absolute token distribution

In [27]:
query = """
    SELECT 
         to_timestamp(bh.timestamp) AS timestamp
        ,MAX(cp.state_root) AS parent_state_root
        ,AVG(total_qa_bytes_power::numeric) * 2^(-50) AS total_power
        ,AVG(total_qa_bytes_committed::numeric) * 2^(-50) as total_committed
        ,AVG(qa_smoothed_position_estimate::numeric) * 2^(-128) * 2^(-50) AS position_estimate
        ,AVG(total_qa_bytes_committed::numeric / total_qa_bytes_power::numeric) as total_committed_over_power
        ,AVG(qa_smoothed_position_estimate::numeric * 2^(-128) / total_qa_bytes_power::numeric) AS position_estimate
        ,AVG(cp.qa_smoothed_velocity_estimate::numeric * 2^(-128) * 2^(-50)) AS velocity_estimate
        

    FROM chain_powers cp
    LEFT JOIN block_headers bh
        ON bh.parent_state_root = cp.state_root
        
    GROUP BY to_timestamp(bh.timestamp)
"""
df = ( pd.read_sql(query, connection) )
df.head()

,timestamp,parent_state_root,total_power,total_committed,position_estimate,total_committed_over_power,position_estimate,velocity_estimate
0,2020-08-24 22:00:00+00:00,bafy2bzacech3yb7xlb7c57v2xh7rvmt4skeidk7z2g36l...,0.703125,0.703125,0.722562,1.0,1.027644,0.003662
1,2020-08-24 22:00:30+00:00,bafy2bzacech3yb7xlb7c57v2xh7rvmt4skeidk7z2g36l...,0.703125,0.703125,0.722562,1.0,1.027644,0.003662
2,2020-08-24 22:01:00+00:00,bafy2bzacecbifligvzk3u5wdigafvubkcxksvy5rwyfhe...,0.703125,0.703125,0.726203,1.0,1.032822,0.003662
3,2020-08-24 22:01:30+00:00,bafy2bzacebpkfyohjp7thdmpjy44lkimctgphlxu3apnk...,0.703125,0.703125,0.729840,1.0,1.037995,0.003662
4,2020-08-24 22:02:30+00:00,bafy2bzacecexoawh5laschovjoelxhnif7sllmt5vzseu...,0.703125,0.703125,0.737105,1.0,1.048327,0.003662


In [22]:
df.shape

(208597, 7)

In [28]:
#meta = MetaData()

#staging_chain_power = Table(
#    'staging_chain_power', meta, 
#    Column('id', Integer, primary_key = True),
#    Column('parent_state_root', String),
#    Column('timestamp', DateTime), 
#    Column('total_power', Numeric), 
#    Column('total_committed', Numeric), 
#    Column('position_estimate', Numeric),
#    Column('total_committed_over_power', Numeric),
#    Column('position_estimate', Numeric),
#    Column('velocity_estimate', Numeric)
# )
#meta.create_all(connection_etl)

In [29]:
df.to_sql(
     name="staging_chain_power"
    ,schema="public"
    ,con=connection_etl
    ,if_exists="append"
    ,index=False
    ,chunksize = 10000
    )

# Chain Rewards

Supports:

- per epoch reward estimate

- per epoch reward velocity estimate

In [13]:
query = """
    SELECT
         MIN( to_timestamp(bh.timestamp) ) AS timestamp
        ,MAX(cr.state_root) AS parent_state_root
        ,AVG( cr.new_reward::numeric * 1e-18 ) AS Per_Epoch_Reward_Actual
        ,AVG( cr.new_reward_smoothed_position_estimate::numeric * 2^(-128) * 1e-18 ) AS Per_Epoch_Reward_Position_Estimate
        
              
        FROM chain_rewards cr
        LEFT JOIN block_headers bh
            ON bh.parent_state_root = cr.state_root
        
        GROUP BY date_trunc('hour', to_timestamp(bh.timestamp))
        
        ORDER BY timestamp ASC
"""
df = ( pd.read_sql(query, connection) )
df.head()

,timestamp,parent_state_root,per_epoch_reward_actual,per_epoch_reward_position_estimate
0,2020-08-24 22:00:00+00:00,bafy2bzacedzff6jdnd662nr3kv5iauhuudz7t5ejvp6vn...,36.271819,36.266576
1,2020-08-24 23:01:00+00:00,bafy2bzacedyw5zbjccw7kcetoxweaxaisq5mmq52m5f5t...,36.270711,36.267119
2,2020-08-25 00:00:00+00:00,bafy2bzacedyner3ipepkv3uycwdgihq7yrac62glhfuhf...,36.268941,36.267388
3,2020-08-25 01:00:00+00:00,bafy2bzacedwygcx3ftm7ayjqcsupvonvivoujahxj4sls...,36.268461,36.267554
4,2020-08-25 02:00:00+00:00,bafy2bzacedzdgbygt7e7gkyxp2tvjcd2txfexcu3xclxe...,36.268070,36.267657


In [14]:
df.shape

(2531, 4)

In [15]:
meta = MetaData()

staging_chain_rewards = Table(
    'staging_chain_rewards', meta, 
    Column('id', Integer, primary_key = True),
    Column('parent_state_root', String),
    Column('timestamp', DateTime), 
    Column('per_epoch_reward_actual', Numeric), 
    Column('per_epoch_reward_position_estimate', Numeric), 
 )
meta.create_all(connection_etl)

In [16]:
df.to_sql(
     name="staging_chain_rewards"
    ,schema="public"
    ,con=connection_etl
    ,if_exists="append"
    ,index=False
    ,chunksize = 10000
    )

In [17]:
query = """
    SELECT 
         COUNT(*)       

    FROM staging_chain_rewards cr
    
"""
df = ( pd.read_sql(query, connection_etl) )
df.head()

,count
0,2531


# Miner Sector Infos

Supports:

- Upcoming Sector expiration

In [60]:
query = """
    SELECT 
         date_trunc( 'hour', to_timestamp( bh.timestamp ) ) AS timestamp
         ,COUNT(CASE WHEN to_timestamp(info.expiration_epoch) > Now() THEN info.expiration_epoch ELSE NULL END) AS upcoming_sector_expiration
         
    FROM miner_sector_infos as info
    LEFT JOIN block_headers bh
        ON bh.parent_state_root = info.state_root
    
    GROUP BY date_trunc( 'hour', to_timestamp( bh.timestamp ) )
"""
df = ( pd.read_sql(query, connection) )
df.tail()

,timestamp,upcoming_sector_expiration
1065,2020-10-15 14:00:00+00:00,0
1066,2020-10-03 12:00:00+00:00,0
1067,2020-10-31 05:00:00+00:00,0
1068,2020-10-26 18:00:00+00:00,0
1069,2020-10-31 20:00:00+00:00,0


In [61]:
df.tail(1)

,timestamp,upcoming_sector_expiration
1069,2020-10-31 20:00:00+00:00,0


In [62]:
meta = MetaData()

staging_miner_sector_infos = Table(
    'staging_miner_sector_infos', meta, 
    Column('id', Integer, primary_key = True),
    Column('timestamp', DateTime), 
    Column('upcoming_sector_expiration', Numeric), 
 )
meta.create_all(connection_etl)

In [63]:
df.tail(1).to_sql(
     name="staging_miner_sector_infos"
    ,schema="public"
    ,con=connection_etl
    ,if_exists="append"
    ,index=False
    ,chunksize = 10000
    )

# Market Deal States

Supports:

- number of deals made

- number of terminated deals

- verified client deals proportion

In [37]:
query = """
    SELECT 
         date_trunc( 'day',  to_timestamp( bh.timestamp ) ) AS timestamp
        ,COUNT( CASE WHEN info.last_update_epoch > 0 THEN deal_id ELSE NULL END ) AS number_of_deals_made
        ,COUNT( CASE WHEN info.slash_epoch > 0 THEN deal_id else NULL END ) AS number_of_terminated_deals
    
    FROM market_deal_states as info
    LEFT JOIN block_headers bh
        ON bh.parent_state_root = info.state_root
        
    WHERE bh.timestamp IS NOT NULL
    
    GROUP BY date_trunc('day',  to_timestamp(bh.timestamp))
"""
df = ( pd.read_sql(query, connection) )
df.head()

,timestamp,number_of_deals_made,number_of_terminated_deals
0,2020-08-24 00:00:00+00:00,4608,0
1,2020-08-25 00:00:00+00:00,4608,0
2,2020-08-26 00:00:00+00:00,27228,0
3,2020-08-27 00:00:00+00:00,50980,0
4,2020-08-28 00:00:00+00:00,92290,0


In [38]:
df.tail()

,timestamp,number_of_deals_made,number_of_terminated_deals
70,2020-11-02 00:00:00+00:00,4969172,274
71,2020-11-03 00:00:00+00:00,5053554,4144
72,2020-11-04 00:00:00+00:00,5092688,6836
73,2020-11-05 00:00:00+00:00,5151038,3622
74,2020-11-06 00:00:00+00:00,919010,18


In [39]:
meta = MetaData()

staging_market_deal_states = Table(
    'staging_market_deal_states', meta, 
    Column('id', Integer, primary_key = True),
    Column('timestamp', DateTime), 
    Column('number_of_deals_made', Numeric), 
    Column('number_of_terminated_deals', Numeric), 
 )
meta.create_all(connection_etl)

In [40]:
df.to_sql(
     name="staging_market_deal_states"
    ,schema="public"
    ,con=connection_etl
    ,if_exists="append"
    ,index=False
    ,chunksize = 10000
    )

# Market Deal Proposals

Supports:

- verified client deals proportion

In [47]:
query = """
    SELECT
         date_trunc('hour',  to_timestamp(bh.timestamp)) AS timestamp
        ,COUNT(mdp.is_verified) filter (where mdp.is_verified::BOOLEAN) / COUNT(mdp.deal_id) AS verified_fraction
              
        FROM market_deal_proposals as mdp
        LEFT JOIN block_headers bh
            ON bh.parent_state_root = mdp.state_root
            
        WHERE bh.timestamp IS NOT NULL
        
        GROUP BY date_trunc('hour',  to_timestamp(bh.timestamp))
"""
df = ( pd.read_sql(query, connection) )
df.head()

,timestamp,verified_fraction
0,2020-08-24 22:00:00+00:00,1
1,2020-08-25 00:00:00+00:00,0
2,2020-08-25 01:00:00+00:00,0
3,2020-08-25 02:00:00+00:00,0
4,2020-08-25 03:00:00+00:00,0


In [48]:
df.tail()

,timestamp,verified_fraction
1687,2020-11-06 00:00:00+00:00,0
1688,2020-11-06 01:00:00+00:00,0
1689,2020-11-06 02:00:00+00:00,0
1690,2020-11-06 03:00:00+00:00,0
1691,2020-11-06 04:00:00+00:00,0


In [49]:
meta = MetaData()

staging_market_deal_proposals = Table(
    'staging_market_deal_proposals', meta, 
    Column('id', Integer, primary_key = True),
    Column('timestamp', DateTime), 
    Column('verified_fraction', Numeric), 
 )
meta.create_all(connection_etl)

In [50]:
df.to_sql(
     name="staging_market_deal_proposals"
    ,schema="public"
    ,con=connection_etl
    ,if_exists="append"
    ,index=False
    ,chunksize = 10000
    )